In [4]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle
import os
import json
from pathlib import Path


## Develop a simple web indexer (pg 43)

In [5]:
class BM25(object):
    def __init__(self, fitted_vectorizer: TfidfVectorizer, b=0.75, k1=1.6):
        self.fitted_vectorizer = fitted_vectorizer
        self.b = b
        self.k1 = k1

    def fit(self, X):
        """Fit IDF to documents X"""
        self.y = super(TfidfVectorizer, self.fitted_vectorizer).transform(X)
        self.avdl = self.y.sum(1).mean()

    def transform(self, q):
        """Calculate BM25 between query q and documents X"""
        b, k1, avdl = self.b, self.k1, self.avdl

        # apply CountVectorizer
        len_y = self.y.sum(1).A1
        (q,) = super(TfidfVectorizer, self.fitted_vectorizer).transform(q)
        assert sparse.isspmatrix_csr(q)

        # convert to csc for better column slicing
        y = self.y.tocsc()[:, q.indices]
        denom = y + (k1 * (1 - b + b * len_y / avdl))[:, None]
        idf = self.fitted_vectorizer._tfidf.idf_[None, q.indices] - 1.0
        numer = y.multiply(np.broadcast_to(idf, y.shape)) * (k1 + 1)
        return (numer / denom).sum(1).A1


In [6]:
def custom_preprocessor(s: str):
    lemmatizer = WordNetLemmatizer()
    s = re.sub(r"[^A-Za-z]", " ", s)
    s = re.sub(r"\s+", " ", s)
    s = re.sub(" +", " ", s)
    s = s.lower()
    s = word_tokenize(s)
    s = set(s).difference(set(stopwords.words("english")))
    s = [word for word in s if len(word) > 2]
    s = [lemmatizer.lemmatize(w) for w in s]
    s = " ".join(s)
    return s


In [7]:
class WebIndexer:
    def __init__(self):
        self.crawled_folder = Path(os.path.abspath('')) / 'crawled/'
        self.stored_file = 'indexer/manual_indexer.pickle'
        if not Path(self.stored_file).parent.exists():
            Path.mkdir(Path(self.stored_file).parent)
        if os.path.isfile(self.stored_file):
            with open(self.stored_file, 'rb') as f:
                cached_dict = pickle.load(f)
            self.__dict__.update(cached_dict)
        else:
            self.run_indexer()
    def run_indexer(self):
        documents = []
        for file in os.listdir(self.crawled_folder):
            if file.endswith(".json"):
                try:
                    j = json.load(open(os.path.join(self.crawled_folder, file)))
                    documents.append(j)
                except json.JSONDecodeError as e:
                    print(file)
        self.documents = pd.DataFrame.from_dict(documents)
        tfidf_vectorizor = TfidfVectorizer(preprocessor=custom_preprocessor, stop_words=stopwords.words('english'))
        tfidf_vectorizor.fit(self.documents.apply(lambda s: ' '.join(s[['title', 'text']]), axis=1))
        self.bm25 = BM25(tfidf_vectorizor)
        self.bm25.fit(self.documents.apply(lambda s: ' '.join(s[['title', 'text']]), axis=1))
        with open(self.stored_file, 'wb') as f:
            pickle.dump(self.__dict__, f)
    def search(self, query):
        scores = self.bm25.transform([query])
        df = pd.DataFrame(scores, columns=["score"])
        return self.documents.join(df)



In [12]:
wi = WebIndexer()
res = wi.search("School")
res[res["score"] > 0].sort_values("score", ascending=False).head(20)


,url,title,text,url_lists,score
10679,https://w3.grad.cmu.ac.th/,"Graduate School, Chiang Mai University",,[],5.454879
10281,https://service.camt.cmu.ac.th/gifted,Gift School 2023,<< คลิกที่นี่ >> ระบบรับสมัคร Gifted School | ...,[https://service.camt.cmu.ac.th/gifted/gifted/...,5.454879
8074,https://service.camt.cmu.ac.th/gifted,Gift School 2023,<< คลิกที่นี่ >> ระบบรับสมัคร Gifted School | ...,[https://service.camt.cmu.ac.th/gifted/gifted/...,5.454879
6541,https://w3.grad.cmu.ac.th/,"Graduate School, Chiang Mai University",,[],5.454879
4670,https://w3.grad.cmu.ac.th/,"Graduate School, Chiang Mai University",,[],5.454879
673,https://www.grad.cmu.ac.th/,"Graduate School, Chiang Mai University",MIdS : Multidisciplinary and Interdisciplinary...,"[https://cmu.to/admission/, https://w3.grad.cm...",5.313894
933,https://www.grad.cmu.ac.th/,"Graduate School, Chiang Mai University",MIdS : Multidisciplinary and Interdisciplinary...,"[https://cmu.to/admission/, https://w3.grad.cm...",5.313894
2395,https://service.camt.cmu.ac.th/gifted/gifted/i...,Gift School 2566,Login Form เลขบัตรประชาชน 13 หลัก ดู P...,[],5.259520
5010,https://service.camt.cmu.ac.th/gifted/gifted/i...,Gift School 2566,Login Form เลขบัตรประชาชน 13 หลัก ดู P...,[],5.259520
4674,https://www.tmc.or.th/,แพทยสภา :: The Medical Council of Thailand,TMC Mail | แผนผังเว็บไซต์ TMC Mail | แผนผัง...,"[https://tmc.or.th/, https://tmc.or.th/En, htt...",5.028001


In [6]:
wi.search("medicine").sort_values("score", ascending=False).drop("url_lists", axis=1).head(20)


,url,title,text,score
4698,https://www.facebook.com/BiochemMedCmu/,Biochemistry Medicine CMU,,2.972423
1600,https://www.facebook.com/BiochemMedCmu/,Biochemistry Medicine CMU,,2.972423
4292,https://www.facebook.com/nationallibraryofmedi...,National Library of Medicine (NLM),,2.956566
8904,https://www.youtube.com/user/NLMNIH,National Library of Medicine - YouTube,เกี่ยวกับ สื่อ ลิขสิทธิ์ ติดต่อเรา ครีเอเตอร์ ...,2.925355
7560,https://w2.med.cmu.ac.th/hcss/topics/news/,ประชาสัมพันธ์ – งานบริการกลางโรงพยาบาล โรงพยาบ...,หน้าหลัก เกี่ยวกับเรา ผู้บริหาร โครงสร้างห...,2.925355
3314,https://www.youtube.com/@emergencycmu,Emergency Medicine CMU - YouTube,เกี่ยวกับ สื่อ ลิขสิทธิ์ ติดต่อเรา ครีเอเตอร์ ...,2.925355
2635,http://www.thaipedlung.org/,สมาคมโรคระบบหายใจและเวชบำบัดวิกฤตในเด็ก :: Ped...,,2.925355
1268,https://www.youtube.com/@emergencycmu,Emergency Medicine CMU - YouTube,เกี่ยวกับ สื่อ ลิขสิทธิ์ ติดต่อเรา ครีเอเตอร์ ...,2.925355
6645,https://www.facebook.com/emcmu/,"EMERGENCY MEDICINE DEPARTMENT, CMU | Chiang Mai",,2.925355
6081,http://www.thaipedlung.org/,สมาคมโรคระบบหายใจและเวชบำบัดวิกฤตในเด็ก :: Ped...,,2.925355


## Elastic search tutorial (pg 50 - 58)
- +oEqEIt7p6lC_=rI1HIC
- eyJ2ZXIiOiI4LjEyLjEiLCJhZHIiOlsiMTcyLjE4LjAuMjo5MjAwIl0sImZnciI6IjNiYmJmMzc3MmY0OWFmMDVlNjZmOTI2NjBjYjUxMWZmZDVkZTY3ODg4M2U0ODgzYTEzNjlmNDEzOTcyYmYwZDQiLCJrZXkiOiJlRE54bkkwQlhKX3FRMUFkSU1FNzpoNzlSSXphV1FIU0pQVWRhRldMckZnIn0=

In [7]:
from elasticsearch import Elasticsearch

es = Elasticsearch("https://localhost:9200", basic_auth=("elastic", "+oEqEIt7p6lC_=rI1HIC"), ca_certs="./http_ca.crt")
es.info()


ObjectApiResponse({'name': '30bfd7881813', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'o_7sMfofS_G5l-SWN2IbUA', 'version': {'number': '8.12.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6185ba65d27469afabc9bc951cded6c17c21e3f3', 'build_date': '2024-02-01T13:07:13.727175297Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
class ElasticIndexer:
    def __init__(self):
        self.crawled_folder = Path(os.path.abspath("")) / "crawled/"
        with open(self.crawled_folder / "url_list.pickle", "rb") as f:
            self.file_mapper = pickle.load(f)
        self.es_client = Elasticsearch(
            "https://localhost:9200",
            basic_auth=("elastic", "+oEqEIt7p6lC_=rI1HIC"),
            ca_certs="./http_ca.crt",
        )

    def run_indexer(self):
        self.es_client.options(ignore_status=400).indices.create(index="simple")
        self.es_client.options(ignore_status=[400, 404]).indices.delete(index="simple")
        for file in os.listdir(self.crawled_folder):
            if file.endswith(".json"):
                try:
                    j = json.load(open(os.path.join(self.crawled_folder, file)))
                    j["id"] = j["url"]
                    self.es_client.index(index="simple", document=j)
                except json.JSONDecodeError as e:
                    print(file)


In [9]:
ei = ElasticIndexer()
ei.run_indexer()


2310681883848955003.json
7886801698163828316.json


In [10]:
query = {"bool": {"must": [{"match": {"text": "apply"}}]}}
results = ei.es_client.search(index="simple", query=query)
print("Got %d Hits:" % results["hits"]["total"]["value"])
for hit in results["hits"]["hits"]:
    print(
        "The title is '{0} ({1})'.".format(
            hit["_source"]["title"], hit["_source"]["url"]
        )
    )


Got 360 Hits:
The title is 'DASHBOARD | Graduate School, Chiang Mai University (https://smart.grad.cmu.ac.th/?p=curr)'.
The title is 'Pricing  |  Compute Engine: Virtual Machines (VMs)  |  Google Cloud (https://cloud.google.com/compute/all-pricing/)'.
The title is 'NON-IMMIGRANT “B” - International Relations Unit (https://w1.med.cmu.ac.th/foreign/en/non-immigrant-b/)'.
The title is 'YouTube TV - Watch & DVR Live Sports, Shows & News (https://tv.youtube.com/?utm_source=gaboutpage&utm_medium=youtubetv&utm_campaign=gabout)'.
The title is 'YouTube TV - Watch & DVR Live Sports, Shows & News (https://tv.youtube.com/welcome/?zipcode=78702)'.
The title is 'YouTube TV - Watch & DVR Live Sports, Shows & News (https://tv.youtube.com/)'.
The title is 'YouTube Research - Home (https://research.youtube/)'.
The title is 'NON-IMMIGRANT “ED” - International Relations Unit (https://w1.med.cmu.ac.th/foreign/en/non-immigrant-ed/)'.
The title is 'FAQ. - CMU - Graduate School (https://www.grad.cmu.ac.th/gra

In [11]:
query = {"regexp": {"text": ".*vision.*"}}
results = ei.es_client.search(index="simple", query=query)
print("Got %d Hits:" % results["hits"]["total"]["value"])
for hit in results["hits"]["hits"]:
    print(
        "The title is '{0} ({1})'.".format(
            hit["_source"]["title"], hit["_source"]["url"]
        )
    )


Got 1159 Hits:
The title is 'ระเบียบและประกาศมหาวิทยาลัยเชียงใหม่ (https://camt.cmu.ac.th/index.php/en/th/serviceview/121b0f9c-32c8-416f-bf9c-47c1dba82af3)'.
The title is 'The Department of Microbiology welcomed visiting students from Biomedical Laboratory Science, UCL University College - ภาควิชาจุลชีววิทยา (https://w1.med.cmu.ac.th/microb/the-department-of-microbiology-welcomed-visiting-students-from-biomedical-laboratory-science-ucl-university-college/)'.
The title is 'Business Intelligence Modernization  |  Google Cloud (https://cloud.google.com/solutions/business-intelligence/)'.
The title is 'History - Faculty of Medicine Chiang Mai University (https://www.med.cmu.ac.th/en/about-us/history/)'.
The title is 'Cloud Composer | Google Cloud (https://cloud.google.com/composer/)'.
The title is 'Automate with AI-powered translation | Google Cloud (https://cloud.google.com/translate/)'.
The title is 'à¸ªà¸³à¸à¸±à¸à¸à¸±à¸à¸à¸²à¸à¸¸à¸à¸ à¸²à¸à¸à¸²à¸£à¸¨à¸¶à¸à¸©à¸² à¸¡à¸«à¸²à¸§à¸´

## manual_index flask app (pg 59)
see indexer_server.py

In [21]:
import requests
import json

query="exam"


In [22]:
json.loads(requests.get(f"http://127.0.0.1:5000/search_es?query={query}").text)


{'elapse': 0.07343411445617676,
 'results': [{'score': 12.686181,
   'text': 'Skip to content    คู่มือสำหรับนักศึกษา หมวดหมู่รวบรวมคู่มือการใช้การอุปกรณ์ต่างๆสำหรับนักศึกษา     ',
   'title': 'คู่มือสำหรับนักศึกษา - ศูนย์พัฒนาเทคโนโลยีแพทยศาสตรศึกษา (MTEC)',
   'url': 'https://w2.med.cmu.ac.th/mtec/topics/manual/manuals-for-students-manual/'},
  {'score': 12.686181,
   'text': 'Skip to content    คู่มือสำหรับนักศึกษา หมวดหมู่รวบรวมคู่มือการใช้การอุปกรณ์ต่างๆสำหรับนักศึกษา     ',
   'title': 'คู่มือสำหรับนักศึกษา - ศูนย์พัฒนาเทคโนโลยีแพทยศาสตรศึกษา (MTEC)',
   'url': 'https://w2.med.cmu.ac.th/mtec/topics/manual/manuals-for-students-manual/'},
  {'score': 11.916158,
   'text': 'Department of Biochemistry  +66-53-935-322  Mon-Fr 08.30-16.30                             Home  Abo',
   'title': 'INTERNATIONAL PROGRAM - Department of Biochemistry',
   'url': 'https://w1.med.cmu.ac.th/biochem/international-program/'},
  {'score': 11.916158,
   'text': 'Department of Biochemistry  +66-53-935-3

In [23]:
json.loads(requests.get(f"http://127.0.0.1:5000/search_manual?query={query}").text)


{'elapse': 0.03786039352416992,
 'results': [{'score': 9.847685939571612,
   'text': 'หน้าหลัก  บุคลากร  หลักสูตร – เกณฑ์แพทยสภา  สำหรับนักศึกษา   คู่มือ นศพ.  กระบวนวิชา   นศพ. ปี 1  นศพ. ปี 2  นศพ. ปี 3    ดาวน์โหลด  ประเมิน    สำหรับอาจารย์-ภาควิชา  ดาวน์โหลด  ติดต่อเรา                      คณะแพทยศาสตร์                                 หน้าหลัก  บุคลากร  หลักสูตร – เกณฑ์แพทยสภา  สำหรับนักศึกษา   คู่มือ นศพ.  กระบวนวิชา   นศพ. ปี 1  นศพ. ปี 2  นศพ. ปี 3    ดาวน์โหลด  ประเมิน    สำหรับอาจารย์-ภาควิชา  ดาวน์โหลด  ติดต่อเรา                         คำสั่ง/ประกาศ คู่มือกระบวนวิชาฯ รายชื่อนักศึกษาแพทย์ ดาวน์โหลด สำหรับอาจารย์-ภาควิชา อาจารย์ผู้สอนประเมินนักศึกษา คำสั่ง/ประกาศ    ⇒ คำสั่งแต่งตั้งคณะกรรมการกระบวนวิชาในหลักสูตรแพทยศาสตร์บัณฑิต ประจำปีการศึกษา 2565       คำสั่งแต่งตั้งคณะกรรมการกระบวนวิชาในหลักสูตรแพทยศาสตรบัณฑิต ระดับชั้นปีที่ 1 ประจำปีการศึกษา 2565  คำสั่งแต่งตั้งคณะกรรมการกระบวนวิชาในหลักสูตรแพทยศาสตรบัณฑิต ระดับชั้นปีที่ 2 ประจำปีการศึกษา 2565  คำสั่งแต่งตั้งคณะกรรมการกระบ

## PageRank tutorial (pg 75-81)

In [27]:
pr.pr_result.sort_values(by='score', ascending=False)


,score
https://w2.med.cmu.ac.th/rau/,0.000275
https://www.med.cmu.ac.th/web/about-medcmu/leadership/,0.000195
https://www.med.cmu.ac.th/web/about-medcmu/facts-and-figures/,0.000194
https://w2.med.cmu.ac.th/rau/backend/,0.000159
https://developers.google.com/youtube,0.000151
...,...
https://www.med.cmu.ac.th/web/#1637725171497-3f694113-9ec1,0.00011
https://www.google.org/,0.00011
https://www.med.cmu.ac.th/nature/volumes/586,0.00011
https://www.med.cmu.ac.th/nature/volumes/374,0.00011
